In [2]:
import pandas as pd
import re

In [3]:
df=pd.read_csv("data//data_original//1113_data.csv")
df_institution=pd.read_csv("data//item_department_match//institution_match.csv")
df_item=pd.read_csv("data//item_department_match//item_cleaned.csv")

In [4]:
df.loc[df["Institution_name"]=="Education.","Institution_name"]="Gauteng - education"

In [5]:
df["Institution_name"]=df["Institution_name"].apply(lambda x:str(x).upper())
df["Institution_name"]=df["Institution_name"].apply(lambda x:re.sub(r"\.","",x))
df_institution["Original_name"]=df_institution["Original_name"].apply(lambda x:x.upper())

In [6]:
df_clean_ins=pd.merge(df,df_institution,how="left",left_on="Institution_name",right_on="Original_name") 

In [7]:
df_use= pd.merge(df_clean_ins,df_item.loc[:,["Item_desp","category"]],how="left",on="Item_desp")
df_use.drop(columns=[" filter","Original_name"],inplace=True)

In [10]:
## because item desp is only cleaned for those orders with payment number>0, we are filtering out other orders
df_use_paid=df_use[df_use["pay_num"]>0]
df_use_paid.reset_index(inplace=True,drop=True)
## aggregate by category
df_agg_cat=df_use_paid.groupby(by=["category"]).pay_num.sum().to_frame()
df_agg_cat.reset_index(inplace=True)
df_agg_cat.rename(columns={"pay_num":"item_sum"},inplace=True)

In [11]:
df_agg_cat.sort_values(by="item_sum",ascending=False,inplace=True)

In [12]:
top_categories=df_agg_cat.head(30)["category"].tolist()

In [14]:
df_agg_cat.head(30)

,category,item_sum
438,Other,5776042799
0,0,2542554102
335,Infrastructure,2309070248
462,PPE001 Surgical masks for patients,1646042983
470,PPE020 Hand sanitisers and disinfectants,1202799159
471,PPE021 Sanitising of buildings,547989962
461,PPE000 Other PPEs,509633763
688,"respirator masks (FFP2, N95)",494812500
463,PPE002 Surgical masks for health care workers,419741242
685,gloves,412359569


In [11]:
df_agg_item_dep=df_use.groupby(by=["category","Cleaned_name"]).agg({'pay_num':'sum','order_num':'sum'})
df_agg_item_dep.reset_index(inplace=True)
df_agg_item_dep=pd.merge(df_agg_item_dep,df_agg_cat,how="left",on="category")
# sort first by catgory, then by department
df_agg_item_dep.sort_values(by=["item_sum","pay_num"],ascending=False,inplace=True)

In [12]:
df_selected=df_agg_item_dep[df_agg_item_dep["category"].isin(top_categories)]
df_selected.drop(columns=["item_sum"],inplace=True)
df_selected.reset_index(inplace=True,drop=True)

C:\Users\hanzh\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
top_three = df_selected.groupby('category').apply(lambda x: x.nlargest(3, 'pay_num')).reset_index(drop=True)  
df_others = df_selected.append(df_selected.merge(top_three,'inner')).drop_duplicates(keep=False)

In [14]:
# Aggregate all other departments except top 3 under each item together
df_others = df_others.groupby('category').agg({'pay_num':'sum','order_num':'sum'}).reset_index()
df_others['Cleaned_name'] = 'Other_departments'
df_viz = pd.concat([top_three,df_others]).sort_values(['category','pay_num'],ascending=False).reset_index(drop=True)

In [15]:
df_viz=pd.merge(df_viz,df_agg_cat,how="left",on="category")
df_viz["percent"]=df_viz["pay_num"]/df_viz["item_sum"]
df_viz

,category,Cleaned_name,pay_num,order_num,item_sum,percent
0,"respirator masks (FFP2, N95)",Gauteng - health,412279310,6.284819e+08,494812500,0.833203
1,"respirator masks (FFP2, N95)",Other_departments,32936176,6.641967e+07,494812500,0.066563
2,"respirator masks (FFP2, N95)",Mpumalanga - health,25821459,3.161302e+07,494812500,0.052184
3,"respirator masks (FFP2, N95)",Kwazulu natal - health,23647555,4.183242e+07,494812500,0.047791
4,gown,Gauteng - health,219317338,5.570439e+08,325121433,0.674571
...,...,...,...,...,...,...
112,Apron,All wcg departments,1741330,6.138687e+05,22579597,0.077120
113,0,Gauteng - health,1951247890,1.285728e+08,2542554102,0.767436
114,0,Public works,213919718,2.148048e+08,2542554102,0.084136
115,0,Other_departments,202291609,1.962667e+08,2542554102,0.079562


In [16]:
pd.set_option('display.max_rows', 200)
df_viz


,category,Cleaned_name,pay_num,order_num,item_sum,percent
0,"respirator masks (FFP2, N95)",Gauteng - health,412279310,6.284819e+08,494812500,0.833203
1,"respirator masks (FFP2, N95)",Other_departments,32936176,6.641967e+07,494812500,0.066563
2,"respirator masks (FFP2, N95)",Mpumalanga - health,25821459,3.161302e+07,494812500,0.052184
3,"respirator masks (FFP2, N95)",Kwazulu natal - health,23647555,4.183242e+07,494812500,0.047791
4,gown,Gauteng - health,219317338,5.570439e+08,325121433,0.674571
5,gown,Other_departments,51033369,1.417745e+08,325121433,0.156967
6,gown,Eastern cape - health,27601300,3.219094e+07,325121433,0.084895
7,gown,Western cape - health,27169426,2.400503e+07,325121433,0.083567
8,gloves,Other_departments,180526434,3.681744e+08,412359569,0.437789
9,gloves,Western cape - health,106030604,1.012795e+08,412359569,0.257131


In [17]:
legend_dep1=df_viz.loc[df_viz["percent"]>0.2,"Cleaned_name"].unique().tolist()

In [18]:
df_topdep_count=df_viz.groupby(by="Cleaned_name").category.count().to_frame()
df_topdep_count.reset_index(inplace=True)

In [19]:
legend_dep2=df_topdep_count.loc[df_topdep_count["category"]>1,"Cleaned_name"].unique().tolist()

In [20]:
selected_departments=list(set(legend_dep1+legend_dep2))

In [21]:
# Manually delete some department with low percentage share only
selected_departments.remove("Eastern cape - health")
selected_departments.remove("Department of correctional services")

In [22]:
df_viz.loc[~df_viz["Cleaned_name"].isin(selected_departments),"Cleaned_name"]='Other_departments'

In [23]:
df_viz_agg=df_viz.groupby(by=["category","Cleaned_name","item_sum"]).agg({'pay_num':'sum','order_num':'sum',"percent":"sum"})
df_viz_agg.reset_index(inplace=True)
df_viz_agg.sort_values(by=["item_sum","percent"],ascending=False,inplace=True)

In [24]:
df_viz_agg["category"].unique()

array(['Other', '0', 'Infrastructure',
       'PPE001 Surgical masks for patients',
       'PPE020 Hand sanitisers and disinfectants',
       'PPE021 Sanitising of buildings', 'PPE000 Other PPEs',
       'respirator masks (FFP2, N95)',
       'PPE002 Surgical masks for health care workers', 'gloves', 'gown',
       'cloth masks', 'Food parcels', 'Non-PPE Covid related items',
       'Thermometer', 'PPE012 Multi purpose sanitisers and disinfectants',
       'Uncategorized masks', 'PPE009 Protective coveralls',
       'PPE019 Surface sanitizers and disinfectants',
       'Visor / Face Shield', 'Renovation', 'dispenser',
       'Human resources-services', 'Telecommunication',
       'Uncategorized sanitisers', 'cleaning services',
       'Other fumigation/disinfecting/decontamination/sanitizing/fogging/deep cleaning',
       'PPE025 Sanitising spray bottles',
       'PPE005 Eye protection - Goggles', 'Apron'], dtype=object)

In [25]:
df_viz_agg["category_short"]=df_viz_agg["category"].apply(lambda x:re.sub("PPE\d\d\d","",x))
df_viz_agg.loc[df_viz_agg["category"]=="Other fumigation/disinfecting/decontamination/sanitizing/fogging/deep cleaning","category_short"]="other fumigation/disinfecting/sanitizing"
df_viz_agg.loc[df_viz_agg["category"]=="0","category_short"]="Category 0"

In [26]:
df_viz_agg

,category,Cleaned_name,item_sum,pay_num,order_num,percent,category_short
23,Other,Department of health,5776042799,3594066219,2.463921e+09,0.622237,Other
25,Other,Other_departments,5776042799,1330738580,1.616761e+09,0.230389,Other
22,Other,All wcg departments,5776042799,433564912,6.501501e+08,0.075063,Other
24,Other,Mpumalanga - health,5776042799,410996796,1.070856e+09,0.071155,Other
1,0,Gauteng - health,2542554102,1951247890,1.285728e+08,0.767436,Category 0
3,0,Public works,2542554102,213919718,2.148048e+08,0.084136,Category 0
2,0,Other_departments,2542554102,202291609,1.962667e+08,0.079562,Category 0
0,0,Department of social development,2542554102,175094885,1.758762e+08,0.068866,Category 0
15,Infrastructure,Gauteng - health,2309070248,886098867,3.526870e+09,0.383747,Infrastructure
16,Infrastructure,Gauteng - infrastructure development,2309070248,573370722,6.641458e+09,0.248312,Infrastructure


In [27]:
df_viz_agg.to_csv("data//data_viz//bar_chart.csv")

In [8]:
df_use.to_csv("data//data_viz//cleaned_full.csv")